Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import matplotlib.pyplot as plt
import time

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
batch_size = 128
num_steps = 3001

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
def run_sgdlr(batch_size, num_steps, learning_rate=0.5, beta=0.1, verbose=False):
    graph = tf.Graph()
    with graph.as_default():
        tf.set_random_seed(1)
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                            shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        # Variables.
        weights = tf.Variable(
            tf.truncated_normal([image_size * image_size, num_labels]))
        biases = tf.Variable(tf.zeros([num_labels]))

        # Training computation.
        logits = tf.matmul(tf_train_dataset, weights) + biases
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
            0.5 * beta * tf.nn.l2_loss(weights)

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(
            tf.matmul(tf_valid_dataset, weights) + biases)
        test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
        
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0 and verbose == True):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      test_accuracy = accuracy(test_prediction.eval(), test_labels)
      if verbose == True:
          print("Test accuracy: %.1f%%" % test_accuracy)
      return test_accuracy
      

In [7]:
def run_nn(batch_size, num_steps, train_x, train_y,
           learning_rate=.1, beta=.1, verbose=False):
    hidden_nodes = 1024

    def weight_variable(shape):
      initial = tf.truncated_normal(shape)
      return tf.Variable(initial)

    def bias_variable(shape):
      initial = tf.zeros(shape)
      return tf.Variable(initial)

    def get_logits(x, w1, b1, w2, b2):
      relu = tf.nn.relu(tf.matmul(x, w1) + b1)
      return (tf.matmul(relu, w2) + b2)

    graph = tf.Graph()
    with graph.as_default():
      tf.set_random_seed(1)
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights_1 = weight_variable([image_size * image_size, hidden_nodes])
      biases_1 = bias_variable([hidden_nodes])

      weights_2 = weight_variable([hidden_nodes, num_labels])
      biases_2 = bias_variable([num_labels])

      # Training computation.
      logits = get_logits(tf_train_dataset, weights_1, biases_1, weights_2, biases_2)
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
            0.5 * beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(get_logits(tf_valid_dataset, weights_1, biases_1, weights_2, biases_2))
      test_prediction = tf.nn.softmax(get_logits(tf_test_dataset, weights_1, biases_1, weights_2, biases_2))

    with tf.Session(graph=graph) as session:
      tf.set_random_seed(1)
      tf.global_variables_initializer().run()
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_y.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_x[offset:(offset + batch_size), :]
        batch_labels = train_y[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0 and verbose):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
        
      acc = accuracy(test_prediction.eval(), test_labels)
      if verbose:
          print("Test accuracy: %.1f%%" % acc)
        
    return acc

In [7]:
start = time.time()
print("SGD Logistic Regression")
print("No L2 Regularization: %.1f%%" % run_sgdlr(batch_size, num_steps, beta=0, verbose=False))
print("With L2 Regularization: %.1f%%" % run_sgdlr(batch_size, num_steps, beta=0.01, verbose=False))
print("Elapsed %.3fs" % (time.time() - start))

SGD Logistic Regression
No L2 Regularization: 86.3%
With L2 Regularization: 88.9%
Elapsed 8.355s


In [8]:
start = time.time()
print("1 hidden-layer neural network")
print("No L2 Regularization: %.1f%%" % run_nn(batch_size, num_steps, train_dataset, train_labels, beta=0))
print("With L2 Regularization: %.1f%%" % run_nn(batch_size, num_steps, train_dataset, train_labels, beta=0.01))    
print("Elapsed %.3fs" % (time.time() - start))

1 hidden-layer neural network
No L2 Regularization: 90.6%
With L2 Regularization: 92.2%
Elapsed 143.045s


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [8]:
def reduce_training_data(divider):
    dataset_size = train_labels.shape[0]
    new_train_dataset = train_dataset[:dataset_size // divider]
    new_train_labels = train_labels[:dataset_size // divider]
    
    return (new_train_dataset, new_train_labels)

new_train_dataset, new_train_labels = reduce_training_data(100)

start = time.time()
print("1 hidden-layer neural network (reduced training data -- overfit example)")
run_nn(batch_size, num_steps, new_train_dataset, new_train_labels, beta=0.01, verbose=True)   
print("Elapsed %.3fs" % (time.time() - start))

1 hidden-layer neural network (reduced training data -- overfit example)
Minibatch loss at step 0: 1895.041748
Minibatch accuracy: 7.0%
Validation accuracy: 21.9%


KeyboardInterrupt: 

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [9]:
def run_nn_with_dropout(batch_size, num_steps, train_x, train_y,
           learning_rate=.1, beta=.1, verbose=False):
    hidden_nodes = 1024

    def weight_variable(shape):
      initial = tf.truncated_normal(shape)
      return tf.Variable(initial)

    def bias_variable(shape):
      initial = tf.zeros(shape)
      return tf.Variable(initial)

    def get_logits(x, w1, b1, w2, b2, keep_prob=.5):
      relu_dropout = tf.nn.dropout(tf.nn.relu(tf.matmul(x, w1) + b1), keep_prob=keep_prob)
      return (tf.matmul(relu_dropout, w2) + b2)

    graph = tf.Graph()
    with graph.as_default():
      tf.set_random_seed(1)
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights_1 = weight_variable([image_size * image_size, hidden_nodes])
      biases_1 = bias_variable([hidden_nodes])

      weights_2 = weight_variable([hidden_nodes, num_labels])
      biases_2 = bias_variable([num_labels])

      # Training computation.
      logits = get_logits(tf_train_dataset, weights_1, biases_1, weights_2, biases_2)
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
            0.5 * beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(get_logits(tf_valid_dataset, weights_1, biases_1, weights_2, biases_2, keep_prob=1))
      test_prediction = tf.nn.softmax(get_logits(tf_test_dataset, weights_1, biases_1, weights_2, biases_2, keep_prob=1))

    with tf.Session(graph=graph) as session:
      tf.set_random_seed(1)
      tf.global_variables_initializer().run()
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_y.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_x[offset:(offset + batch_size), :]
        batch_labels = train_y[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0 and verbose):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
        
      acc = accuracy(test_prediction.eval(), test_labels)
      if verbose:
          print("Test accuracy: %.1f%%" % acc)
        
    return acc

In [11]:
start = time.time()
print("1 hidden-layer neural network (overfit example + dropout)")
run_nn_with_dropout(batch_size, num_steps, new_train_dataset, new_train_labels, beta=0.01, verbose=True)    
print("Elapsed %.3fs" % (time.time() - start))

1 hidden-layer neural network (overfit example + dropout)
Minibatch loss at step 0: 2033.286255
Minibatch accuracy: 10.2%
Validation accuracy: 20.2%
Minibatch loss at step 500: 952.380737
Minibatch accuracy: 96.1%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 576.299255
Minibatch accuracy: 98.4%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 349.284790
Minibatch accuracy: 99.2%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 211.735428
Minibatch accuracy: 100.0%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 128.418076
Minibatch accuracy: 98.4%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 77.877846
Minibatch accuracy: 100.0%
Validation accuracy: 81.3%
Test accuracy: 88.8%
Elapsed 79.504s


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [10]:
start = time.time()
print("1 hidden-layer neural network (all data + dropout)")
run_nn_with_dropout(batch_size, num_steps, train_dataset, train_labels, beta=0.01, verbose=True)    
print("Elapsed %.3fs" % (time.time() - start))

1 hidden-layer neural network (all data + dropout)
Minibatch loss at step 0: 2033.286255
Minibatch accuracy: 10.2%
Validation accuracy: 20.2%
Minibatch loss at step 500: 955.374084
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 1000: 574.996277
Minibatch accuracy: 78.9%
Validation accuracy: 82.6%
Minibatch loss at step 1500: 346.063171
Minibatch accuracy: 83.6%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 209.076599
Minibatch accuracy: 79.7%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 126.581299
Minibatch accuracy: 80.5%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 76.908089
Minibatch accuracy: 81.2%
Validation accuracy: 84.6%
Test accuracy: 91.5%
Elapsed 75.864s


In [11]:
start = time.time()
print("1 hidden-layer neural network (all data + dropout + 6000 steps)")
run_nn_with_dropout(batch_size, 6001, train_dataset, train_labels, beta=0.01, verbose=True)    
print("Elapsed %.3fs" % (time.time() - start))

1 hidden-layer neural network (all data + dropout + 6000 steps)
Minibatch loss at step 0: 2033.286255
Minibatch accuracy: 10.2%
Validation accuracy: 20.2%
Minibatch loss at step 500: 955.374084
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 1000: 574.996277
Minibatch accuracy: 78.9%
Validation accuracy: 82.6%
Minibatch loss at step 1500: 346.063171
Minibatch accuracy: 83.6%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 209.076599
Minibatch accuracy: 79.7%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 126.581299
Minibatch accuracy: 80.5%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 76.908089
Minibatch accuracy: 81.2%
Validation accuracy: 84.6%
Minibatch loss at step 3500: 46.882347
Minibatch accuracy: 84.4%
Validation accuracy: 84.9%
Minibatch loss at step 4000: 28.586117
Minibatch accuracy: 87.5%
Validation accuracy: 85.3%
Minibatch loss at step 4500: 17.571941
Minibatch accuracy: 85.2%
Validation accuracy: 85.0%
Minibatch los

In [23]:
def run_2lnn_with_dropout(batch_size, num_steps, train_x, train_y,
           learning_rate=.1, beta=.1, verbose=False):
    hidden_nodes = 1024

    def weight_variable(shape):
      initial = tf.truncated_normal(shape)
      return tf.Variable(initial)

    def bias_variable(shape):
      initial = tf.zeros(shape)
      return tf.Variable(initial)
    
    def get_layer(previous_layer, w, b, testing=False):
        layer = tf.matmul(previous_layer, w) + b
        if testing:
            return tf.nn.relu(layer)
        else:
            return tf.nn.dropout(tf.nn.relu(layer), keep_prob=.5)

    graph = tf.Graph()
    with graph.as_default():
      tf.set_random_seed(1)
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights_1 = weight_variable([image_size * image_size, hidden_nodes])
      biases_1 = bias_variable([hidden_nodes])
      layer_1_train = get_layer(tf_train_dataset, weights_1, biases_1)
      layer_1_valid = get_layer(tf_valid_dataset, weights_1, biases_1, testing=True)
      layer_1_test = get_layer(tf_test_dataset, weights_1, biases_1, testing=True)

      weights_2 = weight_variable([hidden_nodes, 300])
      biases_2 = bias_variable([300])
      layer_2_train = get_layer(layer_1_train, weights_2, biases_2)
      layer_2_valid = get_layer(layer_1_valid, weights_2, biases_2, testing=True)
      layer_2_test = get_layer(layer_1_test, weights_2, biases_2, testing=True)

      weights_3 = weight_variable([300, num_labels])
      biases_3 = bias_variable([num_labels])
      layer_3_train = get_layer(layer_2_train, weights_3, biases_3)
      layer_3_valid = get_layer(layer_2_valid, weights_3, biases_3, testing=True)
      layer_3_test = get_layer(layer_2_test, weights_3, biases_3, testing=True)
        
      # Training computation.
      logits_train = layer_3_train
      logits_valid = layer_3_valid
      logits_test = layer_3_test
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_train)) + \
            0.5 * beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3))

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits_train)
      valid_prediction = tf.nn.softmax(logits_valid)
      test_prediction = tf.nn.softmax(logits_test)

    with tf.Session(graph=graph) as session:
      tf.set_random_seed(1)
      tf.global_variables_initializer().run()
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_y.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_x[offset:(offset + batch_size), :]
        batch_labels = train_y[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0 and verbose):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
        
      acc = accuracy(test_prediction.eval(), test_labels)
      if verbose:
          print("Test accuracy: %.1f%%" % acc)
        
    return acc

In [24]:
start = time.time()
print("2 hidden-layer neural network (all data + dropout + 6000 steps)")
run_2lnn_with_dropout(batch_size, 6001, train_dataset, train_labels, beta=0.01, verbose=True)    
print("Elapsed %.3fs" % (time.time() - start))

2 hidden-layer neural network (all data + dropout + 6000 steps)
Minibatch loss at step 0: 11705.000977
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 500: 1396.848511
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 848.032227
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Minibatch loss at step 1500: 515.198547
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 313.351074
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 2500: 190.939377
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 3000: 116.702415
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Minibatch loss at step 3500: 71.680847
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 4000: 44.377407
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 4500: 27.819042
Minibatch accuracy: 4.7%
Validation accuracy: 10.0%
Minibatch loss a

--------

No success with 2 hidden layers. Tried various types: 1024-1024-10, 1024-10-10 and 1024-300-10, with learning rates of 0.01, 0.1 and 0.5, and batch size of 128 and 512.

Now I'll try learning rate decay.

---------

In [32]:
def run_nn_with_dropout_and_decay(batch_size, num_steps, train_x, train_y,
           initial_learning_rate=.1, beta=.1, verbose=False):
    hidden_nodes = 1024

    def weight_variable(shape):
      initial = tf.truncated_normal(shape)
      return tf.Variable(initial)

    def bias_variable(shape):
      initial = tf.zeros(shape)
      return tf.Variable(initial)

    def get_logits(x, w1, b1, w2, b2, keep_prob=.5):
      relu_dropout = tf.nn.dropout(tf.nn.relu(tf.matmul(x, w1) + b1), keep_prob=keep_prob)
      return (tf.matmul(relu_dropout, w2) + b2)

    graph = tf.Graph()
    with graph.as_default():
      tf.set_random_seed(1)
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights_1 = weight_variable([image_size * image_size, hidden_nodes])
      biases_1 = bias_variable([hidden_nodes])

      weights_2 = weight_variable([hidden_nodes, num_labels])
      biases_2 = bias_variable([num_labels])

      # Training computation.
      logits = get_logits(tf_train_dataset, weights_1, biases_1, weights_2, biases_2)
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
            0.5 * beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))

      global_step = tf.Variable(1, name='global_step', trainable=False, dtype=tf.int32)
      increment_global_step_op = tf.assign(global_step, global_step+1)
      learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, 1000, 0.5, staircase=True)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
        
      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(get_logits(tf_valid_dataset, weights_1, biases_1, weights_2, biases_2, keep_prob=1))
      test_prediction = tf.nn.softmax(get_logits(tf_test_dataset, weights_1, biases_1, weights_2, biases_2, keep_prob=1))

    with tf.Session(graph=graph) as session:
      tf.set_random_seed(1)
      tf.global_variables_initializer().run()
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_y.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_x[offset:(offset + batch_size), :]
        batch_labels = train_y[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions, _ = session.run(
          [optimizer, loss, train_prediction, increment_global_step_op], feed_dict=feed_dict)
        if (step % 500 == 0 and verbose):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
        
      acc = accuracy(test_prediction.eval(), test_labels)
      if verbose:
          print("Test accuracy: %.1f%%" % acc)
        
    return acc

In [34]:
start = time.time()
print("1 hidden-layer neural network (all data + dropout + decay lr + 6000 steps)")
run_nn_with_dropout_and_decay(batch_size, 6001, train_dataset, train_labels, beta=0.01, initial_learning_rate=0.3, verbose=True)    
print("Elapsed %.3fs" % (time.time() - start))

1 hidden-layer neural network (all data + dropout + decay lr + 6000 steps)
Minibatch loss at step 0: 2033.286255
Minibatch accuracy: 10.2%
Validation accuracy: 26.4%
Minibatch loss at step 500: 349.273743
Minibatch accuracy: 79.7%
Validation accuracy: 81.8%
Minibatch loss at step 1000: 77.683899
Minibatch accuracy: 78.1%
Validation accuracy: 83.9%
Minibatch loss at step 1500: 66.431190
Minibatch accuracy: 86.7%
Validation accuracy: 84.9%
Minibatch loss at step 2000: 57.263008
Minibatch accuracy: 85.9%
Validation accuracy: 85.3%
Minibatch loss at step 2500: 56.447029
Minibatch accuracy: 82.0%
Validation accuracy: 85.2%
Minibatch loss at step 3000: 55.667961
Minibatch accuracy: 82.8%
Validation accuracy: 85.2%
Minibatch loss at step 3500: 55.611835
Minibatch accuracy: 85.9%
Validation accuracy: 85.2%
Minibatch loss at step 4000: 55.550827
Minibatch accuracy: 84.4%
Validation accuracy: 85.2%
Minibatch loss at step 4500: 55.583439
Minibatch accuracy: 84.4%
Validation accuracy: 85.2%
Miniba

Keras is too slow below. SO DON'T RUN!!!!

---------

In [24]:
# setting seed
import numpy as np
np.random.seed(1)

# Keras imports
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.regularizers import l2
from keras.optimizers import SGD

In [62]:
def run_ann_with_keras(batch_size, nb_epoch, X_train, Y_train, beta=.1, sgd_lr=.1, dropout_prob=.5, verbose=0):
    start = time.time()
    
    hidden_nodes = 1024
    image_size = 28
    num_labels = 10
    
    model = Sequential()

    # Input Layer
    model.add(Dense(output_dim=1024, input_dim=image_size*image_size, W_regularizer=l2(beta)))
    model.add(Activation("relu"))
    model.add(Dropout(dropout_prob))
    # Hidden Layer
    model.add(Dense(output_dim=num_labels, W_regularizer=l2(beta)))

    # Output Layer
    model.add(Activation("softmax"))

    # Compile model using categorial crossentropy
    model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01), metrics=['accuracy'])
    
    current_checkpoint = 0
    CHECKPOINT = 300
    while nb_epoch > current_checkpoint * CHECKPOINT:
        initial_epoch = current_checkpoint * CHECKPOINT + 1
        next_epoch = initial_epoch + CHECKPOINT
        print("Epoch: %d | Elapsed time: %.3fs" % (initial_epoch, (time.time() - start)))
        model.fit(X_train, Y_train, nb_epoch=next_epoch, initial_epoch=initial_epoch, batch_size=batch_size, verbose=verbose)
        current_checkpoint += 1

    print("Finished Training |  Elapsed time: %.3fs" % (time.time() - start))
    val_accuracy = model.evaluate(valid_dataset, valid_labels, verbose=verbose)[1]
    print("Validation accuracy: %.2f%% |  Elapsed time: %.3fs" % (val_accuracy * 100.0, (time.time() - start)))
    tst_accuracy = model.evaluate(test_dataset, test_labels, verbose=verbose)[1]
    print("Test accuracy: %.2f%% |  Elapsed time: %.3fs" % (tst_accuracy * 100.0, (time.time() - start)))


In [ ]:
print("1 hidden-layer neural network (overfit example + dropout) KERAS")
run_ann_with_keras(batch_size, num_steps, new_train_dataset, new_train_labels, beta=0.01, verbose=0)


1 hidden-layer neural network (overfit example + dropout) KERAS
Epoch: 1 | Elapsed time: 0.136s


In [ ]:
print("1 hidden-layer neural network (dropout + all data) KERAS")
run_ann_with_keras(batch_size, num_steps, train_dataset, train_labels, beta=0.01, verbose=0)

1 hidden-layer neural network (dropout + all data) KERAS
Epoch: 1 | Elapsed time: 0.143s


In [69]:
def run_ann_with_keras_decay(batch_size, nb_epoch, X_train, Y_train, beta=.1, sgd_lr=.1, dropout_prob=.5, verbose=0):
    start = time.time()
    
    hidden_nodes = 1024
    image_size = 28
    num_labels = 10
    
    model = Sequential()

    # Input Layer
    model.add(Dense(output_dim=1024, input_dim=image_size*image_size, W_regularizer=l2(beta)))
    model.add(Activation("relu"))
    model.add(Dropout(dropout_prob))
    # Hidden Layer
    model.add(Dense(output_dim=num_labels, W_regularizer=l2(beta)))

    # Output Layer
    model.add(Activation("softmax"))

    # Compile model using categorial crossentropy
    model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, decay=1e-6), metrics=['accuracy'])
    
    current_checkpoint = 0
    CHECKPOINT = 300
    while nb_epoch > current_checkpoint * CHECKPOINT:
        initial_epoch = current_checkpoint * CHECKPOINT + 1
        next_epoch = initial_epoch + CHECKPOINT
        print("Epoch: %d | Elapsed time: %.3fs" % (initial_epoch, (time.time() - start)))
        model.fit(X_train, Y_train, nb_epoch=next_epoch, initial_epoch=initial_epoch, batch_size=batch_size, verbose=verbose)
        current_checkpoint += 1

    print("Finished Training |  Elapsed time: %.3fs" % (time.time() - start))
    val_accuracy = model.evaluate(valid_dataset, valid_labels, verbose=verbose)[1]
    print("Validation accuracy: %.2f%% |  Elapsed time: %.3fs" % (val_accuracy * 100.0, (time.time() - start)))
    tst_accuracy = model.evaluate(test_dataset, test_labels, verbose=verbose)[1]
    print("Test accuracy: %.2f%% |  Elapsed time: %.3fs" % (tst_accuracy * 100.0, (time.time() - start)))


In [ ]:
print("1 hidden-layer neural network (dropout + all data + lr_decay) KERAS")
run_ann_with_keras_decay(batch_size, num_steps, train_dataset, train_labels, beta=0.01, verbose=0)


ANSWER
-----

Overall the best results were:
- 1 hidden-layer 784-1024-10 neural network, relu activation
- all data
- dropout=0.5
- l2_loss regularization with beta=0.01
- flat learning rate for SGD=0.1
- epochs=6000
- batch_size=128

Results
- Test accuracy: 92.2%
- Validation accuracy: 85.4%
- Train accuracy: 81.2%
- Time to train: ~154s